In [8]:
import subprocess
import time
import picamera
import numpy as np

MIN_F = 70
MAX_F = 7000
MIN_I = 0
MAX_I = 1785

def avg_color(rgb_array):
    return np.mean(rgb_array, axis=(0, 1)).astype(int)

def color_to_freq(c):
    r, g, b = (((c[k]) >> 4) for k in range(3))  # trim 4 bit from each value
    return 2 * ((r << 8) + (g << 4) + b)

# Make a new process every time we need a new frequency
def sound_proc(freq):
    return subprocess.Popen(['speaker-test', '--frequency', str(freq), '--test', 'sine'])

if __name__ == "__main__":
    try:
        proc = sound_proc(440)  # Initial sound process

        img_output = np.empty((240, 320, 3), dtype=np.uint8)  # This is where we dump the image

        with picamera.PiCamera() as camera:
            camera.resolution = (320, 240)
            time.sleep(2)
            while True:
                camera.capture(img_output, 'rgb')
                proc.kill()  # Kill the old frequency
                freq = color_to_freq(avg_color(img_output))
                print(freq)
                proc = sound_proc(freq)
                time.sleep(0.1)
    except Exception as e:
        print(e)
    finally:
        proc.kill()

3786
3818
3784
4330
3818
4330
2726
3240
2728
2728
2696
2728
2728


KeyboardInterrupt: 